In [32]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd 
import datetime 
import projectq.setups.decompositions as rules

from projectq.backends import Simulator

from projectq.cengines import (AutoReplacer,
                               LocalOptimizer,
                               TagRemover,
                               DecompositionRuleSet)
from projectq.ops import H, Rx, Ry, Rz, Measure, All, CNOT
from projectq import MainEngine
from scipy.optimize import minimize

backend = Simulator(gate_fusion=True)

cache_depth = 10
rule_set = DecompositionRuleSet(modules=[rules])
engines = [TagRemover(), LocalOptimizer(cache_depth), AutoReplacer(rule_set), TagRemover(), LocalOptimizer(cache_depth)
           ]

engine = MainEngine(backend, engines)

# ============================================

from problem2 import *

def normalize(array):
    return array/np.sqrt(np.sum(array**2))

def gen_random_state(n):
    return normalize(np.array([np.random.rand() for i in range(n)]))

def check_solution(param,p,eng):
    n=int(len(param)/3/p)
    param=param.reshape((n,3*p))
    amplitudes=[]

    qureg = eng.allocate_qureg(n)
    All(H) | qureg

    for m in range(p):
        for i in range(n):
            Rx(param[i, 3*m+0]) | qureg[i]
            Ry(param[i, 3*m+1]) | qureg[i]
            Rz(param[i, 3*m+2]) | qureg[i]

        CNOT | (qureg[n-1], qureg[0])
        for i in range(n-1):
            CNOT | (qureg[i], qureg[i+1])

    eng.flush()
    amplitudes=[eng.backend.get_amplitude(conf, qureg) for conf in bitstring(n)]

    All(Measure)|qureg
    return amplitudes

In [23]:
def report(i,N,report_rate=2):
    if (i+1)%report_rate==0:
        print("progress {0}/{1}".format(i+1,N))


In [33]:
def bitstring(n):
    if n > 1:
        return ['0'+bits for bits in bitstring(n-1)]+['1'+bits for bits in bitstring(n-1)]
    elif n == 1:
        return ['0', '1']
    else:
        print('error')

In [29]:
@np.vectorize
def norm(arr):
    return np.sum(arr**2)

In [36]:
def bitstring(n):
    return [bin(i)[2:].zfill(n) for i in range(2**n)]

In [20]:
bitstring(3)

['000', '001', '010', '011', '100', '101', '110', '111']

In [41]:
N=1
length=2**10
p=3
method="COBYLA"



# save the following data
# target state
# generated state
# distance
# optimal parameter
# iteration

t1=datetime.datetime.now()
print(t1)
data=[]
print("starting benchmarking...")
for i in range(N):
    target_state=gen_random_state(length) 
    n = int(np.log2(len(target_state)))
    param = np.ones((n, 3*p)).flatten()

    result=minimize(cost_func,x0=param, args=(target_state,p,engine,None),method=method)
    
    
    cost=result.fun
    optimal_param=result.x
    n_iter=result.nfev
    
    
    gen_state=check_solution(optimal_param,p,engine)
    data.append([target_state,gen_state,optimal_param,cost,n_iter])
    report(i,N,5)

print("writing data...")

t2=datetime.datetime.now()
print(t2)
print(t2-t1)

wrapper=pd.DataFrame(data,columns=["target","result","opt_param","cost","n_iter"])

2019-08-01 01:24:11.927024
starting benchmarking...
writing data...
2019-08-01 01:24:57.356371
0:00:45.429347


In [42]:
wrapper

,target,result,opt_param,cost,n_iter
0,"[0.031187681360220716, 0.026926293922074192, 0...","[(0.008752903327974376-0.035974571098857225j),...","[-0.4858526174372913, 1.7542879696031584, 0.28...",0.145086,1000


In [40]:
wrapper.result[0] #p=1

0    0.131418
Name: cost, dtype: float64

In [43]:
wrapper.result[0] #p=3

[(0.008752903327974376-0.035974571098857225j),
 (0.0002560940351111936-0.037856585397134095j),
 (0.006375512433414724-0.03480164442203874j),
 (0.003619179021491708-0.039821496358935914j),
 (0.006639358984584193-0.04169330700757884j),
 (0.0012941803366963052-0.04005700942897225j),
 (0.0035709690014423326-0.039265252697381556j),
 (0.001549223364855537-0.03788680296526226j),
 (0.003349678961261317-0.034776451542499795j),
 (-0.0027926055707350322-0.03186724725532966j),
 (-0.0005123699912225436-0.032923224555387445j),
 (0.0020774778803815293-0.033982072771828915j),
 (0.003505931471234267-0.038524142595282584j),
 (-0.00032341359665373146-0.040331561794414626j),
 (0.004100037925749419-0.03813168761093744j),
 (-0.0004677336587952455-0.03304942136391149j),
 (0.008069860569676485-0.031785904480218855j),
 (0.00230726304097819-0.029501154579194683j),
 (0.0046595863562161124-0.03044480559820884j),
 (0.005513053189843253-0.03070336271307693j),
 (0.006021988363610387-0.034646943672287045j),
 (0.00175

In [44]:
wrapper.cost #p=3

0    0.145086
Name: cost, dtype: float64